In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing

In [ ]:

!pip install kaggle


In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"anandannapurgv","key":"118a86403f69fa6ac83b26f713511289"}'}

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:

!cp kaggle.json ~/.kaggle/

In [ ]:
!ls ~/.kaggle


kaggle.json


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:

!kaggle datasets download -d kausr25/chatterbotenglish

chatterbotenglish.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/chatterbotenglish.zip

Archive:  /content/chatterbotenglish.zip
replace ai.yml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ai.yml                  
  inflating: botprofile.yml          
  inflating: computers.yml           
  inflating: emotion.yml             
  inflating: food.yml                
  inflating: gossip.yml              
  inflating: greetings.yml           
  inflating: health.yml              
  inflating: history.yml             
  inflating: humor.yml               
  inflating: literature.yml          
  inflating: money.yml               
  inflating: movies.yml              
  inflating: politics.yml            
  inflating: psychology.yml          
  inflating: science.yml             
  inflating: sports.yml              
  inflating: trivia.yml              


In [ ]:
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true -O chatbot_nlp.zip
!unzip chatbot_nlp.zip

--2024-03-03 12:11:58--  https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip [following]
--2024-03-03 12:11:58--  https://github.com/shubham0204/Dataset_Archives/raw/master/chatbot_nlp.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip [following]
--2024-03-03 12:11:58--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/chatbot_nlp.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:

In [ ]:

from tensorflow.keras import preprocessing, utils
import os
import yaml

In [ ]:

dir_path = '/content/chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

In [ ]:
questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( ' ' + answers_with_tags[i] + ' ' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


VOCAB SIZE : 1893


In [ ]:

from gensim.models import Word2Vec
import re

In [ ]:
vocab = []
for word in tokenizer.word_index:
  vocab.append(word)

def tokenize(sentences):
  tokens_list = []
  vocabulary = []
  for sentence in sentences:
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    tokens = sentence.split()
    vocabulary += tokens
    tokens_list.append(tokens)
  return tokens_list, vocabulary


In [ ]:

tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)

(564, 22) 22


In [ ]:
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

(564, 72) 72


In [ ]:
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )


(564, 72, 1893)


In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax )
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 22)]                 0         []                            
                                                                                                  
 input_22 (InputLayer)       [(None, 72)]                 0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 22, 200)              378600    ['input_21[0][0]']            
                                                                                                  
 embedding_5 (Embedding)     (None, 72, 200)              378600    ['input_22[0][0]']            
                                                                                           

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 )
model.save( 'model.keras' )


Epoch 1/150
12/12 [==============================] - 58s 527ms/step - loss: 7.5307
Epoch 2/150
12/12 [==============================] - 7s 533ms/step - loss: 7.0860
Epoch 3/150
12/12 [==============================] - 7s 578ms/step - loss: 6.2466
Epoch 4/150
12/12 [==============================] - 7s 516ms/step - loss: 6.0490
Epoch 5/150
12/12 [==============================] - 7s 606ms/step - loss: 5.9718
Epoch 6/150
12/12 [==============================] - 6s 504ms/step - loss: 5.9475
Epoch 7/150
12/12 [==============================] - 7s 596ms/step - loss: 5.9315
Epoch 8/150
12/12 [==============================] - 6s 499ms/step - loss: 5.9215
Epoch 9/150
12/12 [==============================] - 7s 597ms/step - loss: 5.9046
Epoch 10/150
12/12 [==============================] - 6s 513ms/step - loss: 5.8952
Epoch 11/150
12/12 [==============================] - 7s 588ms/step - loss: 5.8874
Epoch 12/150
12/12 [==============================] - 6s 489ms/step - loss: 5.8816
Epoch 13/150

In [ ]:
def make_inference_models():

    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)

    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)

    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()

    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] )
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word

        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True

        empty_target_seq = np.zeros( ( 1 , 1 ) )
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ]

    print( decoded_translation )

KeyboardInterrupt: Interrupted by user